In [1]:
import pandas as pd
import numpy as np
import datetime as dt

# My packages
from packages.thesis_sequence_algorithm_nominmax import thesis_sequence_algorithm
from packages.thesis_model_nominmax import evaluate_model

# Visualization<
import matplotlib.pyplot as plt
import time
from dateutil.relativedelta import relativedelta


In [2]:
dataset = pd.read_csv('dataset_final_min.csv', infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])
dataset_h = pd.read_csv('dataset_final.csv', infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])

# Thesis Model

- Prepare warm up period (Define quantity)
- Train the ConvLSTM Model by warm-up period initially
- Prepare previous hourly step (Define Quantity)
- Select optimal correlated leg load by the Power Spectrum
- If Correlated leg > Theta
    - Update and reshape the input vector for ConvLSTM model
    - Implement single step ahead forecast
    - Retrain the ConvLSTM model by the observed hourly load of previous predicted time step
- Else 
    - Implement persistence model
    - Retrain the ConvLSTM model by the observed hourly load of previous predicted time step 


score_PSConvLSTM

# Month Test

In [3]:
dataset_h = dataset_h.loc[:,['Global_active_power','tempC','weekday','time']]
df = dataset_h[((dataset_h.index >= '2009-03-01') & (dataset_h.index < '2009-06-01'))|((dataset_h.index >= '2009-12-01') & (dataset_h.index < '2010-01-01'))]

In [66]:
dict_list_lags = dict({'1_lags': 1,
                        '2_lags': 2,
                        '3_lags': 3,
                        '4_lags': 4,
                        '5_lags': 5,
                        '6_lags': 6,
                        '12_lags': 12,
                        '24_lags': 24})
                        
dict_list_theta = dict({'90%': 0.5929527277,
                        '80%': 0.5995907576,
                        '70%': 0.6108522834,
                        '60%': 0.6255697670,
                        '50%': 0.6342439183})
for test in range(0,5):
    for lags, quantaslags in zip(dict_list_lags.values(),dict_list_lags.keys()):
        for theta_threshold,percentage_lags in zip(dict_list_theta.values(),dict_list_theta.keys()):
                start_date = '2009-04-01'
                end_date = '2009-05-01'
                dataset_filtered = df[(df.index >= start_date) & (df.index < end_date)]
                autocorrelation_type = 'acf'
                algorithm = 'CLSAF-ACF'
                warm_up_time = 168
                leg_days_time = 168
                n_steps = 1
                n_length = lags
                theta_threshold = theta_threshold
                n_input = n_steps * n_length
                n_features = df.shape[1]
                n_out = 1
                epochs = 50
                epochs_retrain = 1
                batch_size = 1
                filters = 36
                activation = 'relu'
                loss = 'mse'
                optimizer = 'adam'
                karnel = (1,2)
                dense_1 = 4
                dense_2 = 1
                model_parameters_PSConvLSTM = thesis_sequence_algorithm(dataset_filtered, 
                                                                                autocorrelation_type, 
                                                                                warm_up_time, 
                                                                                theta_threshold, 
                                                                                leg_days_time, 
                                                                                n_steps, 
                                                                                n_length, 
                                                                                n_input, 
                                                                                n_features, 
                                                                                n_out, 
                                                                                epochs, 
                                                                                batch_size, 
                                                                                filters, 
                                                                                activation, 
                                                                                loss, 
                                                                                optimizer,
                                                                                karnel,
                                                                                epochs_retrain,
                                                                                dense_1,
                                                                                dense_2)
                #RUN ALGO
                start_time = time.time()
                predict_list_PSConvLSTM, observed_list_PSConvLSTM = model_parameters_PSConvLSTM.master_algorithm()
                scores_PSConvLSTM = evaluate_model(predict_list_PSConvLSTM, observed_list_PSConvLSTM)
                score_PSConvLSTM, scores_PSConvLSTM = scores_PSConvLSTM.evaluate_model()
                time_run = (time.time() - start_time)
                # Read results csv
                teste = pd.read_csv('params_test.csv', index_col= 'Unnamed: 0')
                # Register actual version
                resgister_version = {'start_date': start_date, 'end_date': end_date, 'warm_up_time': warm_up_time, 'leg_days' : leg_days_time,
                                    'theta_threshold': theta_threshold, 'Percentageoflags': percentage_lags ,'n_length': n_length, 'epochs':epochs, 'filters':filters, 
                                    'epochs_retrain': epochs_retrain, 'dense_1': dense_1, 'karnels' : [karnel], 
                                    'score': score_PSConvLSTM, 'max_loss': np.max(scores_PSConvLSTM), 'time_run': time_run}
                # Create Data Frame
                resgister_version_df = pd.DataFrame(data = resgister_version, index = [algorithm] )
                # Concat and save
                pd.concat([teste, resgister_version_df]).to_csv('params_test.csv')
teste = pd.read_csv('params_test.csv', index_col= 'Unnamed: 0')
teste.tail(10)

10.772675037384033
43.454594373703
11.831488609313965
36.76049208641052
12.003093957901001
38.630651235580444
13.460535764694214
37.54465293884277
11.994195461273193
33.36064839363098
11.351137161254883
40.36345076560974
11.662259817123413
41.47500133514404
12.510384321212769
36.53213906288147
12.211210250854492
36.63594937324524
12.166281938552856
36.11372780799866
13.968856573104858
42.98361158370972
11.70591926574707
39.382872104644775
12.101959466934204
37.91503381729126
11.783125162124634
36.02427792549133
12.464736700057983
34.92629027366638
12.713282585144043
42.32080960273743
11.916654825210571
41.25785422325134
11.981842517852783
38.89310622215271
12.193090200424194
36.33340358734131
12.778966188430786
33.40436768531799
11.481374025344849
41.83240842819214
11.59738039970398
40.420310974121094
11.983474254608154
39.307485580444336
12.903779029846191
37.82718253135681
12.371425151824951
33.58443260192871
12.052287817001343
41.45210838317871
12.344965696334839
40.26334810256958
1

,start_date,end_date,warm_up_time,leg_days,theta_threshold,leg_days_time,n_length,epochs,filters,epochs_retrain,dense_1,karnels,score,max_loss,time_run,Percentageoflags
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.050982,NaN,12.0,50.0,36.0,1.0,4.0,"(1, 2)",0.523978,2.691423,53.159236,90%
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.057665,NaN,12.0,50.0,36.0,1.0,4.0,"(1, 2)",0.558408,3.741425,54.648834,80%
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.063902,NaN,12.0,50.0,36.0,1.0,4.0,"(1, 2)",0.531558,3.419546,48.028606,70%
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.069820,NaN,12.0,50.0,36.0,1.0,4.0,"(1, 2)",0.560147,3.546509,46.497933,60%
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.076930,NaN,12.0,50.0,36.0,1.0,4.0,"(1, 2)",0.999089,5.098300,44.081678,50%
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.050982,NaN,24.0,50.0,36.0,1.0,4.0,"(1, 2)",0.582616,4.290523,53.002832,90%
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.057665,NaN,24.0,50.0,36.0,1.0,4.0,"(1, 2)",0.561457,3.224199,51.138626,80%
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.063902,NaN,24.0,50.0,36.0,1.0,4.0,"(1, 2)",0.599336,3.835724,49.519445,70%
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.069820,NaN,24.0,50.0,36.0,1.0,4.0,"(1, 2)",1.072029,5.133966,47.863124,60%
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.076930,NaN,24.0,50.0,36.0,1.0,4.0,"(1, 2)",0.549928,3.580159,45.153602,50%


In [5]:
teste = pd.read_csv('params_test.csv', index_col= 'Unnamed: 0')

In [9]:
teste = teste[teste['Percentageoflags'].notna()]
teste = teste[teste['leg_days'] == 168]
teste = teste[teste['score'] > 0]
teste.sort_values('score').head(20)

,start_date,end_date,warm_up_time,leg_days,theta_threshold,leg_days_time,n_length,epochs,filters,epochs_retrain,dense_1,karnels,score,max_loss,time_run,Percentageoflags
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.057665,NaN,6.0,50.0,36.0,1.0,4.0,"(1, 2)",0.511842,2.674460,52.610309,80%
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.000000,NaN,6.0,60.0,36.0,1.0,4.0,"(1, 2)",0.514277,2.476953,54.156770,90%
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.000000,NaN,5.0,60.0,36.0,1.0,4.0,"(1, 2)",0.515015,2.658088,69.750262,5_lags
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.000000,NaN,12.0,80.0,36.0,1.0,4.0,"(1, 2)",0.515809,2.789189,62.500996,90%
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.063902,NaN,12.0,50.0,36.0,1.0,4.0,"(1, 2)",0.516173,3.148123,46.001537,70%
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.050982,NaN,6.0,50.0,36.0,1.0,4.0,"(1, 2)",0.516255,2.650820,51.012523,90%
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.050982,NaN,6.0,50.0,36.0,1.0,4.0,"(1, 2)",0.517109,2.851267,53.505394,90%
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.057665,NaN,6.0,50.0,36.0,1.0,4.0,"(1, 2)",0.517580,2.769004,48.654298,80%
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.050982,NaN,3.0,50.0,36.0,1.0,4.0,"(1, 2)",0.517965,2.637703,50.287398,90%
CLSAF-Power Cepstrum,2009-04-01,2009-05-01,168.0,168.0,0.000000,NaN,1.0,60.0,36.0,1.0,4.0,"(1, 2)",0.518051,2.681683,69.656631,1_lags


# Min step

In [6]:
dataset = dataset.loc[:,['Global_active_power','tempC','weekday','time']]
df_min = dataset[(dataset.index >= '2009-04-01') & (dataset.index < '2009-05-01')]
df_min

,Global_active_power,tempC,weekday,time
datetime,,,,
2009-04-01 00:00:00,0.626,6,0,0
2009-04-01 00:01:00,0.472,6,0,0
2009-04-01 00:02:00,0.468,6,0,0
2009-04-01 00:03:00,0.538,6,0,0
2009-04-01 00:04:00,1.152,6,0,0
...,...,...,...,...
2009-04-30 23:55:00,2.056,8,0,23
2009-04-30 23:56:00,1.458,8,0,23
2009-04-30 23:57:00,1.416,8,0,23


In [35]:
list_theta = [0.966]
for i in list_theta:
        start_date = '2009-04-01'
        end_date = '2009-05-01'
        dataset_filtered = df_min[(df_min.index >= start_date) & (df_min.index < end_date)]
        autocorrelation_type = 'acf'
        algorithm = 'CLSAF_OneMinute (Adapted with autoregressive lag being t-1)'
        warm_up_time = 10080
        theta_threshold = i
        leg_days_time = 10080
        n_steps = 1
        n_length = 1
        n_input = n_steps * n_length
        n_features = df_min.shape[1]
        n_out = 1
        epochs = 20
        epochs_retrain = 1
        batch_size = 1
        filters = 36
        activation = 'relu'
        loss = 'mse'
        optimizer = 'adam'
        karnel = (1,2)
        dense_1 = 4
        dense_2 = 1
        model_parameters_PSConvLSTM = thesis_sequence_algorithm(dataset_filtered, 
                                                                        autocorrelation_type, 
                                                                        warm_up_time, 
                                                                        theta_threshold, 
                                                                        leg_days_time, 
                                                                        n_steps, 
                                                                        n_length, 
                                                                        n_input, 
                                                                        n_features, 
                                                                        n_out, 
                                                                        epochs, 
                                                                        batch_size, 
                                                                        filters, 
                                                                        activation, 
                                                                        loss, 
                                                                        optimizer,
                                                                        karnel,
                                                                        epochs_retrain,
                                                                        dense_1,
                                                                        dense_2)
        #RUN ALGO
        start_time = time.time()
        predict_list_PSConvLSTM, observed_list_PSConvLSTM = model_parameters_PSConvLSTM.master_algorithm()
        scores_PSConvLSTM = evaluate_model(predict_list_PSConvLSTM, observed_list_PSConvLSTM)
        score_PSConvLSTM, scores_PSConvLSTM = scores_PSConvLSTM.evaluate_model()
        time_run = (time.time() - start_time)
        # Read results csv
        teste = pd.read_csv('params_test.csv', index_col= 'Unnamed: 0')
        # Register actual version
        resgister_version = {'start_date': start_date, 'end_date': end_date, 'warm_up_time': warm_up_time, 'leg_days' : leg_days_time,
                            'theta_threshold': theta_threshold, 'leg_days_time': leg_days_time ,'n_length': n_length, 'epochs':epochs, 'filters':filters, 
                            'epochs_retrain': epochs_retrain, 'dense_1': dense_1, 'karnels' : [karnel], 
                            'score': score_PSConvLSTM, 'max_loss': np.max(scores_PSConvLSTM), 'time_run': time_run}
        # Create Data Frame
        resgister_version_df = pd.DataFrame(data = resgister_version, index = [algorithm] )
        # Concat and save
        pd.concat([teste, resgister_version_df]).to_csv('params_test.csv')

334.80764746665955
2130.886918783188


In [24]:
dataset_filtered['Global_active_power'][10079:-1]

datetime
2009-04-07 23:59:00    0.498
2009-04-08 00:00:00    0.500
2009-04-08 00:01:00    0.498
2009-04-08 00:02:00    0.486
2009-04-08 00:03:00    0.404
                       ...  
2009-04-30 23:54:00    2.058
2009-04-30 23:55:00    2.056
2009-04-30 23:56:00    1.458
2009-04-30 23:57:00    1.416
2009-04-30 23:58:00    1.420
Name: Global_active_power, Length: 33120, dtype: float64

# Persistence model

In [25]:
scores_persistence = evaluate_model(dataset_filtered['Global_active_power'][10079:-1], np.array(observed_list_PSConvLSTM))
score_persistence, scores_persistence_s = scores_persistence.evaluate_model()

In [26]:
score_persistence

0.2252362489448635

In [32]:
np.array(scores_persistence_s).max()

3.3599999999999994

In [18]:
start_date = '2009-01-01'
end_date = '2009-02-01'
dataset_filtered = df_min[(df_min.index >= start_date) & (df_min.index < end_date)]
teste = dataset_filtered['Global_active_power']

In [19]:
teste_2 = teste[10080:]

In [20]:
predicted = teste_2[:-1]
observed = teste_2[1:]

In [21]:
scores_PSConvLSTM = evaluate_model(predicted, observed)
score_PSConvLSTM, scores_PSConvLSTM = scores_PSConvLSTM.evaluate_model()
score_PSConvLSTM

0.21017019955390953

In [9]:
results = pd.read_csv('params_test - Copy (3).csv', sep = ';')
# Transform dates in datetime
results['start_date'] = pd.to_datetime(results['start_date'])
results_hour = results[results['leg_days'] != 10080]
results_hour.head(1)

,Unnamed: 0,start_date,end_date,warm_up_time,leg_days,theta_threshold,leg_days_time,n_length,epochs,filters,epochs_retrain,dense_1,karnels,score,max_loss,time_run
0,Persistence,2008-01-09,01/10/2008 00:00,168.0,168.0,40000.0,168.0,1.0,20.0,36.0,1.0,4.0,"(1, 2)",0.659419,3.136967,29.759267


In [12]:
# Create pivot table to analyze mean score
results_normal = results_hour
analysis_theta = pd.DataFrame(results_normal.groupby(['start_date','theta_threshold','Unnamed: 0']).min()['score']).pivot_table(values = 'score',index='start_date',columns = ['theta_threshold','Unnamed: 0'])
analysis_theta.rename(columns = {1000.0 : 'ConvLSTM', 40000.0 : 'Persistence'}, inplace = True)
analysis_theta.iloc[:,:].mean().sort_values()

theta_threshold  Unnamed: 0
500.0            ConvLSTM      0.210515
4.9e-05          PSConvLSTM    0.264356
4.8e-05          PSConvLSTM    0.281310
4.75e-05         PSConvLSTM    0.314015
4.7e-05          PSConvLSTM    0.355569
                                 ...   
0.0023           PSConvLSTM    0.678159
1000.0           ConvLSTM      0.739936
16.5             PSConvLSTM    0.819854
16.9             PSConvLSTM    1.077071
4.5e-05          PSConvLSTM    1.256246
Length: 82, dtype: float64

In [15]:
analysis_theta['None'].mean()

Unnamed: 0
Prophet    0.602488
dtype: float64